In [4]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
# things we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import pickle
import random

In [5]:
intents = pd.read_json('intents.json')

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rohitemmadishetty/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

45 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
82 unique stemmed words ["'s", ',', 'a', 'advers', 'al', 'anyon', 'ar', 'awesom', 'be', 'behavy', 'blood', 'by', 'bye', 'can', 'caus', 'chat', 'check', 'could', 'dat', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'giv', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hist', 'hospit', 'how', 'i', 'id', 'is', 'lat', 'list', 'load', 'loc', 'log', 'look', 'lookup', 'man', 'me', 'mod', 'nearby', 'next', 'nic', 'of', 'off', 'op', 'paty', 'pharm', 'press', 'provid', 'react', 'rel', 'result', 'search', 'see', 'show', 'suit', 'support', 'task', 'thank', 'that', 'ther', 'til', 'tim', 'to', 'transf', 'up', 'want', 'what', 'which', 'with', 'you']


In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

In [8]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [9]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [10]:
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
45/45 [==============================] - 0s 5ms/step - loss: 2.2364 - acc: 0.1111
Epoch 2/200
45/45 [==============================] - 0s 301us/step - loss: 2.2008 - acc: 0.1333
Epoch 3/200
45/45 [==============================] - 0s 275us/step - loss: 2.0572 - acc: 0.2222
Epoch 4/200
45/45 [==============================] - 0s 307us/step - loss: 1.9735 - acc: 0.3111
Epoch 5/200
45/45 [==============================] - 0s 314us/step - loss: 1.8693 - acc: 0.2889
Epoch 6/200
45/45 [==============================] - 0s 348us/step - loss: 1.8815 - acc: 0.3556
Epoch 7/200
45/45 [==============================] - 0s 323us/step - loss: 1.6684 - acc: 0.5778
Epoch 8/200
45/45 [==============================] - 0s 378us/step - loss: 1.4580 - acc: 0.5778
Epoch 9/200
45/45 [==============================] - 0s 368us/step - loss: 1.3695 - acc: 0.6667
Epoch 10/200
45/45 [==============================] - 0s 442us/step - loss: 1.3581 - acc: 0.6444
Epoch 11/200
45/45 [=====================

45/45 [==============================] - 0s 264us/step - loss: 0.0306 - acc: 1.0000
Epoch 86/200
45/45 [==============================] - 0s 241us/step - loss: 0.0254 - acc: 1.0000
Epoch 87/200
45/45 [==============================] - 0s 215us/step - loss: 0.0255 - acc: 1.0000
Epoch 88/200
45/45 [==============================] - 0s 290us/step - loss: 0.0295 - acc: 1.0000
Epoch 89/200
45/45 [==============================] - 0s 238us/step - loss: 0.0397 - acc: 1.0000
Epoch 90/200
45/45 [==============================] - 0s 253us/step - loss: 0.0394 - acc: 1.0000
Epoch 91/200
45/45 [==============================] - 0s 259us/step - loss: 0.0487 - acc: 0.9778
Epoch 92/200
45/45 [==============================] - 0s 279us/step - loss: 0.0699 - acc: 0.9778
Epoch 93/200
45/45 [==============================] - 0s 264us/step - loss: 0.0194 - acc: 1.0000
Epoch 94/200
45/45 [==============================] - 0s 253us/step - loss: 0.0572 - acc: 0.9778
Epoch 95/200
45/45 [=======================

45/45 [==============================] - 0s 243us/step - loss: 0.0065 - acc: 1.0000
Epoch 169/200
45/45 [==============================] - 0s 247us/step - loss: 0.0093 - acc: 1.0000
Epoch 170/200
45/45 [==============================] - 0s 245us/step - loss: 0.0290 - acc: 1.0000
Epoch 171/200
45/45 [==============================] - 0s 257us/step - loss: 0.0117 - acc: 1.0000
Epoch 172/200
45/45 [==============================] - 0s 239us/step - loss: 0.0138 - acc: 1.0000
Epoch 173/200
45/45 [==============================] - 0s 262us/step - loss: 0.0138 - acc: 1.0000
Epoch 174/200
45/45 [==============================] - 0s 271us/step - loss: 0.0088 - acc: 1.0000
Epoch 175/200
45/45 [==============================] - 0s 273us/step - loss: 0.0077 - acc: 1.0000
Epoch 176/200
45/45 [==============================] - 0s 267us/step - loss: 0.0056 - acc: 1.0000
Epoch 177/200
45/45 [==============================] - 0s 235us/step - loss: 0.0101 - acc: 1.0000
Epoch 178/200
45/45 [=============

In [11]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [12]:
p = bow("Load blood pessure for patient", words)
print (p)
print (classes)

found in bag: load
found in bag: blood
found in bag: for
found in bag: paty
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]
['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [13]:
# Use pickle to load in the pre-trained model
import tensorflow as tf

global graph
graph = tf.get_default_graph()
with open(f'chat-assistant-model.pkl', 'rb') as f:
    model = pickle.load(f)

In [14]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

In [15]:
p = classify_local("Can you help me login to the dashboard ?")
print (p)

found in bag: can
found in bag: you
found in bag: help
found in bag: me
found in bag: to
[('options', '0.9999881')]


In [16]:
from flask import Flask, request, redirect, jsonify
import pandas as pd

app = Flask(__name__)
@app.route("/katana-ml/api/v1.0/assistant", methods=['POST'])
def classify():
    ERROR_THRESHOLD = 0.25
    
    sentence = request.json['sentence']
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    # return tuple of intent and probability
    
    response = jsonify(return_list)
    return response
# running REST interface, port=5000 for direct test, port=5001 for deployment from PM2
if __name__ == "__main__":
    app.run(debug=False, host='0.0.0.0', port=5001)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Apr/2019 11:18:44] "POST /katana-ml/api/v1.0/assistant HTTP/1.1" 200 -


found in bag: hello
found in bag: how
found in bag: ar
found in bag: you


127.0.0.1 - - [30/Apr/2019 11:20:28] "POST /katana-ml/api/v1.0/assistant HTTP/1.1" 200 -


found in bag: to
found in bag: a


...

found in bag: is


127.0.0.1 - - [30/Apr/2019 11:28:40] "POST /katana-ml/api/v1.0/assistant HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2019 11:28:48] "POST /katana-ml/api/v1.0/assistant HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2019 11:29:00] "POST /katana-ml/api/v1.0/assistant HTTP/1.1" 200 -


found in bag: you
found in bag: ar


127.0.0.1 - - [30/Apr/2019 11:29:38] "POST /katana-ml/api/v1.0/assistant HTTP/1.1" 200 -


found in bag: hello
found in bag: how
found in bag: ar
found in bag: you
